In [2]:
import os
from typing import Tuple
import util.config_loader as cfg
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers
from tensorflow.keras import losses

batch_size = 32
seed = 42
max_features = 10000
sequence_length = 250
embedding_dim = 16
url = cfg.get('data_dir_name')


In [3]:
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(os.path.join(url, 'train'),
                                                                    batch_size=batch_size,
                                                                    validation_split=0.2,
                                                                    subset='training',
                                                                    seed=seed)

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(os.path.join(url, 'train'),
                                                                batch_size=batch_size,
                                                                validation_split=0.2,
                                                                subset='training',
                                                                seed=seed)

raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(os.path.join(url, 'test'),
                                                                    batch_size=batch_size,
                                                                    seed=seed)


Found 6298 files belonging to 13 classes.
Using 5039 files for training.
Found 6298 files belonging to 13 classes.
Using 5039 files for training.
Found 3892 files belonging to 13 classes.


In [4]:
def custom_standardization(input_data):
  print(input_data)
  return input_data


vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [5]:
vectorize_layer.get_vocabulary()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x93 in position 0: invalid start byte